In [22]:
import cv2
import csv
import time
from basefunctions import *
from imageprocessing import *
from matching.matching_contextclass import MatchContextClass
from matching.method1 import MatchingMethod1
from preprocessing.preprocessing_contextclass import PreprocessContextClass
from preprocessing.preprocess_granulation1 import GranulationWithThreshold
from preprocessing.preprocess_granulation2 import GranulationStartedFromMin
from preprocessing.preprocess_simpleremove import SimpleRemoving

In [23]:
features = cv2.ORB_create(500)

image1 = cv2.imread('etalonimages//sparrow3.jpg')
image1gray = makeImgGray(image1)
image2 = cv2.imread('etalonimages//synychka1.jpg')
image2gray = makeImgGray(image2)
image3 = cv2.imread('etalonimages//lastivka1.jpg')
image3gray = makeImgGray(image3)
image4 = cv2.imread('etalonimages//pigeon5.jpg')
image4gray = makeImgGray(image4)
image5 = cv2.imread('etalonimages//raven1.jpg')
image5gray = makeImgGray(image5)

print ("   sparrow")
des2 = findKeypoints(image1gray, features)
bit_des_etalon1 = convertAllDescriptors(des2)

print ("   synychka")
des2 = findKeypoints(image2gray, features)
bit_des_etalon2 = convertAllDescriptors(des2)

print ("   lastivka")
des2 = findKeypoints(image3gray, features)
bit_des_etalon3 = convertAllDescriptors(des2)

print ("   pigeon")
des2 = findKeypoints(image4gray, features)
bit_des_etalon4 = convertAllDescriptors(des2)

print ("   raven")
des2 = findKeypoints(image5gray, features)
bit_des_etalon5 = convertAllDescriptors(des2)

bit_des_etalons = [bit_des_etalon1, bit_des_etalon2, bit_des_etalon3, bit_des_etalon4, bit_des_etalon5]

   sparrow
num of descriptors: 493
   synychka
num of descriptors: 482
   lastivka
num of descriptors: 485
   pigeon
num of descriptors: 481
   raven
num of descriptors: 492


In [24]:
image6 = cv2.imread('notetalonimages//duck2.jpg')
image6gray = makeImgGray(image6)
image7 = cv2.imread('notetalonimages//jay2.jpg')
image7gray = makeImgGray(image7)
image8 = cv2.imread('notetalonimages//starling3.jpg')
image8gray = makeImgGray(image8)

originalimages = [image1gray, image2gray, image3gray, image4gray, image5gray, image6gray, image7gray, image8gray]

In [25]:
method1 = MatchingMethod1()
matcher = MatchContextClass(method1)

In [26]:
granulation1 = GranulationWithThreshold()
imgpreprocessor = PreprocessContextClass(granulation1)
reduced_descset_etalons1 = imgpreprocessor.descriptors_preprocessing(bit_des_etalons)
reduced_descset_etalons1 = imgpreprocessor.descriptors_preprocessing(reduced_descset_etalons1)

granulation2 = GranulationStartedFromMin()
imgpreprocessor.set_strategy(granulation2)
reduced_descset_etalons2 = imgpreprocessor.descriptors_preprocessing(bit_des_etalons)
reduced_descset_etalons2 = imgpreprocessor.descriptors_preprocessing(reduced_descset_etalons2)

In [27]:
preprocessing3 = SimpleRemoving()
imgpreprocessor.set_strategy(preprocessing3)
reduced_descset_etalons3 = imgpreprocessor.descriptors_preprocessing(bit_des_etalons)
reduced_descset_etalons3 = imgpreprocessor.descriptors_preprocessing(reduced_descset_etalons3)
# print("reduced_descset_etalons2")
# for etalondesc in reduced_descset_etalons2:
#     print(len(etalondesc))

In [28]:
allimages = []
for img in originalimages:
    originalimage = img
    rotatedimg1 = rotate_20_degrees(img)
    rotatedimg2 = rotate_minus20_degrees(img)
    resizedimg1 = scale_up(img)
    resizedimg2 = scale_down(img)
    rotated1resized1 = scale_up(rotatedimg1)
    rotated1resized2 = scale_down(rotatedimg1)
    rotated2resized1 = scale_up(rotatedimg2)
    rotated2resized2 = scale_down(rotatedimg2)
    imagesoptions = [originalimage, rotatedimg1, rotatedimg2,
                      resizedimg1, resizedimg2,
                     rotated1resized1, rotated1resized2, rotated2resized1, rotated2resized2]
    allimages.append(imagesoptions)

In [29]:
results = []
timeres = []
for i in range(len(allimages)):
    imageresults = []
    timeresfor1img = []
    for j in range(len(allimages[0])):
        method_results = []
        methodtimeres = []
        print("image " + str(i) + " " + str(j))
        des1 = findKeypoints(allimages[i][j], features)
        bit_des_img = convertAllDescriptors(des1)

        # print("without granulation")
        matcher.set_strategy(method1)
        start = time.time()
        result_method1 = matcher.match_images(bit_des_img, bit_des_etalons)
        end = time.time()
        methodtimeres.append(end-start)
        method_results.append(result_method1)
        
        # print("granulation with threshold")
        start = time.time()
        result_granulation1method1 = matcher.match_images(bit_des_img, reduced_descset_etalons1)
        end = time.time()
        methodtimeres.append(end-start)
        method_results.append(result_granulation1method1)

        # print("granulation by distance")
        start = time.time()
        result_granulation2method1 = matcher.match_images(bit_des_img, reduced_descset_etalons2)
        end = time.time()
        methodtimeres.append(end-start)
        method_results.append(result_granulation2method1)

        # print("simple removing")
        start = time.time()
        result_simpleremoving = matcher.match_images(bit_des_img, reduced_descset_etalons3)
        end = time.time()
        methodtimeres.append(end-start)
        method_results.append(result_simpleremoving)
        
        imageresults.append(method_results)
        # timeresfor1img.append(methodtimeres)
    results.append(imageresults)
    print(imageresults)
    # timeres.append(timeresfor1img)

image 0 0
num of descriptors: 493
image 0 1
num of descriptors: 500
image 0 2
num of descriptors: 500
image 0 3
num of descriptors: 500
image 0 4
num of descriptors: 456
image 0 5
num of descriptors: 500
image 0 6
num of descriptors: 496
image 0 7
num of descriptors: 500
image 0 8
num of descriptors: 500
[[[493, 0, 0, 0, 0], [374, 16, 33, 15, 24], [348, 28, 22, 9, 9], [303, 31, 29, 18, 23]], [[422, 22, 16, 7, 18], [311, 40, 39, 24, 33], [287, 46, 32, 10, 32], [247, 49, 30, 23, 38]], [[436, 22, 13, 6, 15], [330, 39, 37, 21, 22], [310, 38, 23, 15, 26], [266, 53, 31, 16, 32]], [[457, 15, 3, 6, 7], [333, 30, 32, 23, 23], [316, 33, 18, 14, 19], [272, 36, 31, 20, 32]], [[412, 13, 8, 6, 12], [296, 26, 37, 26, 28], [300, 35, 23, 13, 20], [249, 36, 29, 21, 41]], [[412, 29, 9, 9, 22], [308, 36, 33, 19, 42], [288, 40, 31, 18, 31], [243, 56, 26, 20, 34]], [[374, 33, 21, 18, 28], [292, 47, 40, 23, 30], [282, 42, 35, 18, 32], [222, 56, 34, 20, 45]], [[416, 32, 3, 8, 17], [301, 45, 25, 25, 27], [270,

In [30]:
accuracies = []
for img in results:
    img_accuracies = []
    for transformedimg in img:
        transformedimg_accuracies = []
        for method in transformedimg:
            method_accuracies = []
            # print(method)
            maxelem1st = max(method)
            maxelem2nd = 0
            for elem in method:
                if(elem>maxelem2nd and elem != maxelem1st):
                    maxelem2nd = elem
            accuracy = (maxelem1st-maxelem2nd)/sum(method)
            transformedimg_accuracies.append(round(accuracy, 3))
        img_accuracies.append(transformedimg_accuracies)
    accuracies.append(img_accuracies)
    
print(accuracies)

[[[1.0, 0.738, 0.769, 0.673], [0.825, 0.606, 0.592, 0.512], [0.841, 0.648, 0.66, 0.535], [0.906, 0.683, 0.708, 0.604], [0.885, 0.627, 0.678, 0.553], [0.796, 0.607, 0.608, 0.493], [0.719, 0.567, 0.587, 0.44], [0.807, 0.605, 0.618, 0.549], [0.738, 0.521, 0.605, 0.43]], [[1.0, 0.798, 0.831, 0.713], [0.861, 0.727, 0.702, 0.667], [0.915, 0.748, 0.748, 0.725], [0.889, 0.708, 0.714, 0.65], [0.937, 0.776, 0.756, 0.75], [0.802, 0.667, 0.644, 0.638], [0.793, 0.661, 0.634, 0.643], [0.83, 0.674, 0.634, 0.677], [0.804, 0.664, 0.633, 0.651]], [[1.0, 0.845, 0.807, 0.757], [0.947, 0.768, 0.753, 0.777], [0.931, 0.762, 0.749, 0.714], [0.904, 0.739, 0.667, 0.64], [0.866, 0.737, 0.701, 0.726], [0.838, 0.693, 0.663, 0.587], [0.791, 0.674, 0.642, 0.632], [0.854, 0.681, 0.66, 0.546], [0.793, 0.622, 0.648, 0.638]], [[1.0, 0.851, 0.82, 0.751], [0.905, 0.783, 0.751, 0.66], [0.897, 0.766, 0.716, 0.688], [0.951, 0.771, 0.735, 0.676], [0.926, 0.824, 0.776, 0.699], [0.849, 0.714, 0.691, 0.644], [0.791, 0.697, 0.717

In [31]:
accuracies2d = [item for sublist in accuracies for item in sublist]
print(accuracies2d)

[[1.0, 0.738, 0.769, 0.673], [0.825, 0.606, 0.592, 0.512], [0.841, 0.648, 0.66, 0.535], [0.906, 0.683, 0.708, 0.604], [0.885, 0.627, 0.678, 0.553], [0.796, 0.607, 0.608, 0.493], [0.719, 0.567, 0.587, 0.44], [0.807, 0.605, 0.618, 0.549], [0.738, 0.521, 0.605, 0.43], [1.0, 0.798, 0.831, 0.713], [0.861, 0.727, 0.702, 0.667], [0.915, 0.748, 0.748, 0.725], [0.889, 0.708, 0.714, 0.65], [0.937, 0.776, 0.756, 0.75], [0.802, 0.667, 0.644, 0.638], [0.793, 0.661, 0.634, 0.643], [0.83, 0.674, 0.634, 0.677], [0.804, 0.664, 0.633, 0.651], [1.0, 0.845, 0.807, 0.757], [0.947, 0.768, 0.753, 0.777], [0.931, 0.762, 0.749, 0.714], [0.904, 0.739, 0.667, 0.64], [0.866, 0.737, 0.701, 0.726], [0.838, 0.693, 0.663, 0.587], [0.791, 0.674, 0.642, 0.632], [0.854, 0.681, 0.66, 0.546], [0.793, 0.622, 0.648, 0.638], [1.0, 0.851, 0.82, 0.751], [0.905, 0.783, 0.751, 0.66], [0.897, 0.766, 0.716, 0.688], [0.951, 0.771, 0.735, 0.676], [0.926, 0.824, 0.776, 0.699], [0.849, 0.714, 0.691, 0.644], [0.791, 0.697, 0.717, 0.602

In [32]:
with open('output.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(accuracies2d)